In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os
import re
import string
import gzip
import shutil
import glob
from datetime import datetime
import matplotlib.pyplot as plt
from spellchecker import SpellChecker
%matplotlib inline

/home/rm1667/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


### Emails Data

In [2]:
emails_data = pd.read_csv('Email/emails_raw.csv')

In [3]:
emails_2022 = emails_data[emails_data['Year'] == 2022]

In [4]:
emails_2022.shape

(80100, 13)

In [ ]:
emails_df = emails_2022[['Name', 'Subject','Body']]

In [ ]:
emails_df.head()

,Name,Subject,Body
34715,DUE @ 11:59 PM | Upset the Setup,re: President Joe Biden,☆ OFFICIAL UTS APPROVAL POLL | REGISTERED TO P...
34863,Coach Tuberville,🎉 Happy New Year 🎉,"96 Peter, The countdown is on – 2022 is quickl..."
34872,Derek Kilmer for Congress,One last update,"Peter, Before tonight's festivities begin, we ..."
34873,Sarah Longwell,A great year for democracy,We thought you might want to see this special ...
34875,Donald J. Trump,2022 is almost over,--- | | | | | | | Untitled Document![Trump 202...


In [ ]:
def pre_processing(text):
    if isinstance(text, str):
        # Lowercasing
        text = text.lower()

        # Removing HTML tags
        def remove_html_tags(text):
            pattern = re.compile('<.*?>')
            return pattern.sub(r'', text)

        text = remove_html_tags(text)

        # Removing URLs
        def remove_url(text):
            pattern = re.compile(r'https?://\S+|www\.\S+')
            return pattern.sub(r'', text)

        text = remove_url(text)

        # Remove punctuations
        punc = string.punctuation
        def remove_punc(text):
            return text.translate(str.maketrans('', '', punc))

        text = remove_punc(text)
        
        # Spelling corrections
        spell = SpellChecker()

        def correct_spelling(text):
            words = text.split()
            corrected_words = []
            for word in words:
                correction = spell.correction(word)
                corrected_words.append(correction if correction else word)
            return ' '.join(corrected_words)

        text = correct_spelling(text)

    return text

In [ ]:
emails_df['Body'] = emails_df['Body'].apply(pre_processing)

In [ ]:
emails_df['Subject'] = emails_df['Subject'].apply(pre_processing)

In [ ]:
emails_df